# Ohio 2008 Presidential Elections: Data Cleaning & Preprocessing

**Goal:** Build a clean, analysis-ready county-level table for Ohio, 2008 by merging the presidential primary and presidential general election results, then derive summary stats (party totals) and information-theoretic measures (entropy).

**Output**: A single CSV where each row is a county and columns include:

- Primary per-candidate vote counts (prefixed with `pri_`)
- General per-candidate vote counts (prefixed with `gen_`)
- Party totals: `rep_primary_total`, `dem_primary_total`, `rep_general_total`, `dem_general_total`
- Entropy features: `rep_primary_entropy`, `dem_primary_entropy`

**Last Updated**: 2025/09/18

## 0. Library Import

In [41]:
import re
import pandas as pd
import numpy as np
from pathlib import Path

## 1. Inputs & Parameters

Define raw file paths once here so the entire notebook is easy to rerun on another machine. If a path changes, we only update it here. We keep a single `OUTPUT_PATH` so all exports land in one known place

In [4]:
# OH 2008 dataset path
PRIMARY_PATH = r"../../data/raw/2008/OH/20080304__oh__primary.csv"
GENERAL_PATH = r"../../data/raw/2008/OH/20081104__oh__general.csv"

# Output directory
OUTPUT_PATH  = r"../../data/processed/2008/OH/"

# Analysis parameters
DISPLAY_ROWS = 10   # Number of rows to display in dataframes

## 2. Load & Filter

We load primary and general datasets separately and immediately subset to the rows we truly need:

- Restrict `office` to 'President' to avoid mixing down-ballot contests
- Restrict `party` to Democratic and Republican 

We also remove columns that are fully missing or irrelevant post-filter (e.g., a district column that’s empty for county-level rows)

### a. Primary Election Dataset

In [5]:
# Load primary data
primary_df = pd.read_csv(PRIMARY_PATH)
primary_df.head(DISPLAY_ROWS)

,county,candidate,votes,party,office,district
0,Butler,Hillary Clinton,2143,Democratic,President,1
1,Hamilton,Hillary Clinton,40123,Democratic,President,1
2,Butler,John Edwards,52,Democratic,President,1
3,Hamilton,John Edwards,816,Democratic,President,1
4,Butler,Barack Obama,1139,Democratic,President,1
5,Hamilton,Barack Obama,74021,Democratic,President,1
6,Adams,Hillary Clinton,2829,Democratic,President,2
7,Brown,Hillary Clinton,5487,Democratic,President,2
8,Clermont,Hillary Clinton,16488,Democratic,President,2
9,Hamilton,Hillary Clinton,19311,Democratic,President,2


In [6]:
# Different values in 'office' column
primary_df["office"].value_counts()

office
President               904
US Representative       477
State Representative    366
State Senate            139
Name: count, dtype: int64

In [7]:
# Only keep rows where 'office' is 'President'
primary_df = primary_df[primary_df["office"] == "President"]

In [8]:
# Now, drop the "office" column as it's no longer needed. Also, drop the district column
primary_df = primary_df.drop(columns=["office", "district"]).reset_index(drop=True)
primary_df.head(DISPLAY_ROWS)

,county,candidate,votes,party
0,Butler,Hillary Clinton,2143,Democratic
1,Hamilton,Hillary Clinton,40123,Democratic
2,Butler,John Edwards,52,Democratic
3,Hamilton,John Edwards,816,Democratic
4,Butler,Barack Obama,1139,Democratic
5,Hamilton,Barack Obama,74021,Democratic
6,Adams,Hillary Clinton,2829,Democratic
7,Brown,Hillary Clinton,5487,Democratic
8,Clermont,Hillary Clinton,16488,Democratic
9,Hamilton,Hillary Clinton,19311,Democratic


In [9]:
# Value counts for all columns in primary_df
primary_df.nunique()

county        88
candidate      8
votes        745
party          2
dtype: int64

In [10]:
# Unique parties in primary_df
primary_df["party"].value_counts()

party
Republican    565
Democratic    339
Name: count, dtype: int64

In [11]:
# Candidates in primary_df
primary_df["candidate"].value_counts()

candidate
Hillary Clinton    113
John Edwards       113
Barack Obama       113
Mike Huckabee      113
John McCain        113
Ron Paul           113
Mitt Romney        113
Fred Thompson      113
Name: count, dtype: int64

In [12]:
# Missing values count
primary_df.isnull().sum()

county       0
candidate    0
votes        0
party        0
dtype: int64

In [13]:
# Final look at the cleaned primary_df
primary_df.head(DISPLAY_ROWS)

,county,candidate,votes,party
0,Butler,Hillary Clinton,2143,Democratic
1,Hamilton,Hillary Clinton,40123,Democratic
2,Butler,John Edwards,52,Democratic
3,Hamilton,John Edwards,816,Democratic
4,Butler,Barack Obama,1139,Democratic
5,Hamilton,Barack Obama,74021,Democratic
6,Adams,Hillary Clinton,2829,Democratic
7,Brown,Hillary Clinton,5487,Democratic
8,Clermont,Hillary Clinton,16488,Democratic
9,Hamilton,Hillary Clinton,19311,Democratic


### b. General Election Dataset

In [14]:
# Load general data
general_df = pd.read_csv(GENERAL_PATH)
general_df.head(DISPLAY_ROWS)

,candidate,county,district,office,party,votes
0,Donald Allen,Adams,NaN,President,Independent,0
1,Donald Allen,Allen,NaN,President,Independent,0
2,Donald Allen,Ashland,NaN,President,Independent,0
3,Donald Allen,Ashtabula,NaN,President,Independent,0
4,Donald Allen,Athens,NaN,President,Independent,0
5,Donald Allen,Auglaize,NaN,President,Independent,0
6,Donald Allen,Belmont,NaN,President,Independent,0
7,Donald Allen,Brown,NaN,President,Independent,0
8,Donald Allen,Butler,NaN,President,Independent,0
9,Donald Allen,Carroll,NaN,President,Independent,0


In [15]:
# Unique value counts for all columns in general_df
general_df.nunique()

candidate     288
county         88
district       99
office          5
party           7
votes        1352
dtype: int64

In [16]:
general_df["party"].value_counts()

party
Independent     847
Republican      494
Democratic      475
Libertarian     110
Green           109
Constitution     88
Socialist        88
Name: count, dtype: int64

In [17]:
# We will only keep Democratic and Republican parties for consistency
general_df = general_df[general_df["party"].isin(["Democratic", "Republican"])]
general_df["party"].value_counts()

party
Republican    494
Democratic    475
Name: count, dtype: int64

In [18]:
# Different values in 'office' column
general_df["office"].value_counts()

office
State Representative    282
US Representative       226
President               176
Attorney General        176
State Senate            109
Name: count, dtype: int64

In [19]:
# We will only keep rows where 'office' is 'President'
general_df = general_df[general_df["office"] == "President"]
general_df = general_df.drop(columns="office")

In [20]:
# Different candidates in general_df
general_df["candidate"].value_counts()

candidate
John McCain     88
Barack Obama    88
Name: count, dtype: int64

In [21]:
# Missing values count
general_df.isnull().sum()

candidate      0
county         0
district     176
party          0
votes          0
dtype: int64

In [22]:
# Given the dataset now has 176 rows, and district has 176 missing values, we will drop the column
general_df = general_df.drop(columns=["district"]).reset_index(drop=True)

In [23]:
# Final look at cleaned general_df
general_df.head(DISPLAY_ROWS)

,candidate,county,party,votes
0,John McCain,Adams,Republican,6914
1,John McCain,Allen,Republican,29940
2,John McCain,Ashland,Republican,15158
3,John McCain,Ashtabula,Republican,18949
4,John McCain,Athens,Republican,9742
5,John McCain,Auglaize,Republican,16414
6,John McCain,Belmont,Republican,15422
7,John McCain,Brown,Republican,12192
8,John McCain,Butler,Republican,105341
9,John McCain,Carroll,Republican,7097


## 3. Table Pivoting

We convert tall (one row per county/party/candidate) into wide (one row per county with one column per candidate). This creates the consistent schema with previous group cleaned data.

Helper functions:

- `normalize_party(s)`: maps common forms (e.g., “Democratic”, “Republican”) to keys dem/rep so column names are stable
- `candidate_token(name)`: turns “Barack Obama” -> OBAMA, “John McCain” -> MCCAIN, etc. Create a short, readable, unique token for column names
- `pivot_wide(df, prefix, key_col="county")`: Main pivot function
        
    * groups by `county` x `party` × `candidate`, sums `votes`,
    * pivots to columns named like:
        * Primary: `pri_dem_OBAMA`, `pri_rep_MCCAIN`,...
        * General: `gen_dem_OBAMA`, `gen_rep_MCCAIN`,...

    * flattens the MultiIndex into plain column strings,
    * returns one wide row per county

In [24]:
def normalize_party(s: pd.Series) -> pd.Series:
    """
    Normalize party names: Democratic -> dem, Republican -> rep
    """
    return(s.str.strip()
           .str.capitalize()
           .map({"Democratic": "dem", "Republican": "rep"})
           .fillna(s.str.strip().str.lower()))      # For defensive purposes only, would not expect other parties

In [25]:
def cand_token(name: str) -> str:
    """
    Turn 'Hillary Clinton' -> 'CLINTON', 'John McCain' -> 'MCCAIN'.
    Removes punctuation; uses last token.
    """
    if pd.isna(name):
        return "UNKNOWN"
    t = re.sub(r"[^A-Za-z0-9\s]+", "", str(name)).strip().upper().split()
    return t[-1] if t else "UNKNOWN"

In [26]:
def candidate_token(name: str) -> str:
    """
    Turn John McCain -> MCCAIN, Barack Obama -> OBAMA
    Keep last name/token, capitalize, and remove punctuation
    """
    if pd.isna(name):
        return "UNKNOWN"                # Defensive purposes only, would not expect missing values
    
    # Remove punctuation and split by whitespace
    t = re.sub(r"[^A-Za-z0-9\s]+", "", str(name)).strip().upper().split()
    return t[-1] if t else "UNKNOWN"    # Return last token or UNKNOWN if empty

In [27]:
def pivot_wide(df: pd.DataFrame, prefix: str, key_col: str="county") -> pd.DataFrame:
    """
    Pivot the dataframe to wide format based on party and candidate
    """
    # Normalize party names
    df['party_key'] = normalize_party(df['party'])
    
    # Create candidate tokens
    df['candidate_token'] = df['candidate'].apply(candidate_token)
    
    # Create new column names based on party and candidate token
    df['new_col'] = prefix + '_' + df['party'] + '_' + df['candidate_token']
    
    # Pivot the dataframe
    pivot_df = df.pivot_table(index=key_col, 
                              columns=["party_key", "candidate_token"], 
                              values="votes", 
                              aggfunc='sum', 
                              fill_value=0)
    
    # Flatten multi-level columns
    pivot_df.columns = [f"{prefix}_{p}_{c}" for p, c in pivot_df.columns]
    
    # Reset index to turn key_col back into a column
    pivot_df = pivot_df.reset_index()
    
    return pivot_df

In [28]:
# Primary dataframe pivot
primary_pivot = pivot_wide(primary_df, prefix="pri")
primary_pivot.head(DISPLAY_ROWS)

,county,pri_dem_CLINTON,pri_dem_EDWARDS,pri_dem_OBAMA,pri_rep_HUCKABEE,pri_rep_MCCAIN,pri_rep_PAUL,pri_rep_ROMNEY,pri_rep_THOMPSON
0,Adams,2829,145,829,1482,1492,75,104,67
1,Allen,7200,350,6801,4067,6813,848,738,323
2,Ashland,3913,153,2587,2687,3342,544,338,180
3,Ashtabula,13231,412,6375,2769,3818,682,329,214
4,Athens,7590,362,7396,1018,1771,408,108,71
5,Auglaize,3008,185,2085,1742,2615,490,282,124
6,Belmont,14204,932,4636,1283,2175,132,201,213
7,Brown,5487,277,1947,1945,2307,199,177,91
8,Butler,27599,589,22824,13758,19825,1179,1807,845
9,Carroll,3663,232,1731,1308,1839,275,166,103


In [29]:
# Primary dataframe shape after pivot
primary_pivot.shape

(88, 9)

In [30]:
# General dataframe pivot
general_pivot = pivot_wide(general_df, prefix="gen")
general_pivot.head(DISPLAY_ROWS)

,county,gen_dem_OBAMA,gen_rep_MCCAIN
0,Adams,4170,6914
1,Allen,19522,29940
2,Ashland,9300,15158
3,Ashtabula,25027,18949
4,Athens,20722,9742
5,Auglaize,6738,16414
6,Belmont,16302,15422
7,Brown,7503,12192
8,Butler,66030,105341
9,Carroll,6423,7097


In [31]:
# General dataframe shape after pivot
general_pivot.shape

(88, 3)

## 4. Merge Dataframes

Before merging, we verify that county names match across primary and general:

In [42]:
# Check if county names match between primary_df and general_df
primary_counties = set(primary_df["county"].unique())
general_counties = set(general_df["county"].unique())
common_counties = primary_counties.intersection(general_counties)
print(f"Number of common counties: {len(common_counties)} out of {len(primary_counties)}")

Number of common counties: 88 out of 88


Great. Since we know that all counties name are matched, we don't need to perform further data preprocessing to match the county names. Thus, we can now merge them:

In [33]:
# Merge primary and general dataframes on 'county'
merged_df = primary_pivot.merge(general_pivot, on="county", how="inner").fillna(0)    # There should be no missing values to fill with 0
merged_df.head(DISPLAY_ROWS)

,county,pri_dem_CLINTON,pri_dem_EDWARDS,pri_dem_OBAMA,pri_rep_HUCKABEE,pri_rep_MCCAIN,pri_rep_PAUL,pri_rep_ROMNEY,pri_rep_THOMPSON,gen_dem_OBAMA,gen_rep_MCCAIN
0,Adams,2829,145,829,1482,1492,75,104,67,4170,6914
1,Allen,7200,350,6801,4067,6813,848,738,323,19522,29940
2,Ashland,3913,153,2587,2687,3342,544,338,180,9300,15158
3,Ashtabula,13231,412,6375,2769,3818,682,329,214,25027,18949
4,Athens,7590,362,7396,1018,1771,408,108,71,20722,9742
5,Auglaize,3008,185,2085,1742,2615,490,282,124,6738,16414
6,Belmont,14204,932,4636,1283,2175,132,201,213,16302,15422
7,Brown,5487,277,1947,1945,2307,199,177,91,7503,12192
8,Butler,27599,589,22824,13758,19825,1179,1807,845,66030,105341
9,Carroll,3663,232,1731,1308,1839,275,166,103,6423,7097


In [34]:
# Statistics check on merged dataframe 
merged_df.describe()

,pri_dem_CLINTON,pri_dem_EDWARDS,pri_dem_OBAMA,pri_rep_HUCKABEE,pri_rep_MCCAIN,pri_rep_PAUL,pri_rep_ROMNEY,pri_rep_THOMPSON,gen_dem_OBAMA,gen_rep_MCCAIN
count,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000
mean,14313.863636,446.954545,11997.375000,3850.431818,6482.420455,632.840909,619.772727,296.625000,33409.590909,30429.772727
std,23125.139942,551.272299,27847.418446,4287.015541,8787.733791,788.487464,906.853379,376.866369,67712.943302,41173.882455
min,1487.000000,72.000000,458.000000,351.000000,397.000000,39.000000,36.000000,34.000000,2463.000000,3021.000000
25%,3495.750000,158.500000,1788.000000,1570.250000,2232.500000,199.000000,174.500000,106.000000,6504.750000,9605.750000
50%,5482.500000,249.000000,2888.500000,2398.500000,3211.500000,324.000000,275.500000,162.000000,11663.000000,15418.500000
75%,14060.000000,449.000000,9030.250000,4206.000000,6324.000000,789.000000,708.750000,327.000000,26607.750000,33471.250000
max,160053.000000,3512.000000,190902.000000,23929.000000,47142.000000,4594.000000,5153.000000,2468.000000,458422.000000,218486.000000


Now, we will add party totals columns: 

- Primary totals:
    * `rep_primary_total` = sum of all `pri_rep_*` columns
    * `dem_primary_total` = sum of all `pri_dem_*` columns

- General totals:
    * `rep_general_total` = sum of all `gen_rep_*` columns
    * `dem_general_total` = sum of all `gen_dem_*` columns

In [35]:
# Add party totals
rep_primary_cols   = [c for c in merged_df.columns if c.startswith("pri_rep_")]
dem_primary_cols   = [c for c in merged_df.columns if c.startswith("pri_dem_")]
rep_general_cols   = [c for c in merged_df.columns if c.startswith("gen_rep_")]
dem_general_cols   = [c for c in merged_df.columns if c.startswith("gen_dem_")]

merged_df["rep_primary_total"] = merged_df[rep_primary_cols].sum(axis=1) if rep_primary_cols else 0
merged_df["dem_primary_total"] = merged_df[dem_primary_cols].sum(axis=1) if dem_primary_cols else 0
merged_df["dem_general_total"] = merged_df[dem_general_cols].sum(axis=1) if dem_general_cols else 0
merged_df["rep_general_total"] = merged_df[rep_general_cols].sum(axis=1) if rep_general_cols else 0

In [36]:
# Preview merged dataframe with totals
merged_df.head(DISPLAY_ROWS)

,county,pri_dem_CLINTON,pri_dem_EDWARDS,pri_dem_OBAMA,pri_rep_HUCKABEE,pri_rep_MCCAIN,pri_rep_PAUL,pri_rep_ROMNEY,pri_rep_THOMPSON,gen_dem_OBAMA,gen_rep_MCCAIN,rep_primary_total,dem_primary_total,dem_general_total,rep_general_total
0,Adams,2829,145,829,1482,1492,75,104,67,4170,6914,3220,3803,4170,6914
1,Allen,7200,350,6801,4067,6813,848,738,323,19522,29940,12789,14351,19522,29940
2,Ashland,3913,153,2587,2687,3342,544,338,180,9300,15158,7091,6653,9300,15158
3,Ashtabula,13231,412,6375,2769,3818,682,329,214,25027,18949,7812,20018,25027,18949
4,Athens,7590,362,7396,1018,1771,408,108,71,20722,9742,3376,15348,20722,9742
5,Auglaize,3008,185,2085,1742,2615,490,282,124,6738,16414,5253,5278,6738,16414
6,Belmont,14204,932,4636,1283,2175,132,201,213,16302,15422,4004,19772,16302,15422
7,Brown,5487,277,1947,1945,2307,199,177,91,7503,12192,4719,7711,7503,12192
8,Butler,27599,589,22824,13758,19825,1179,1807,845,66030,105341,37414,51012,66030,105341
9,Carroll,3663,232,1731,1308,1839,275,166,103,6423,7097,3691,5626,6423,7097


Finally, we compute Shannon entropy of each party's primary vote distribution by candidate within the county:

In [37]:
# Function to calculate the shannon entropy
def shannon_entropy(df: pd.DataFrame, cols: list[str]) -> pd.Series:
    # Defensive check for empty cols list
    if not cols:
        return pd.Series(0.0, index=df.index)
    
    # Calculate probabilities
    probabilities = df[cols].div(df[cols].sum(axis=1).replace(0, np.nan), axis=0)       # Avoid division by zero
    entropy = -(probabilities * np.log(probabilities)).sum(axis=1)
    return entropy.replace([np.inf, -np.inf, np.nan], 0.0)                              # Replace inf and NaN with 0.0

In [38]:
# Apply shannon entropy calculation
merged_df["rep_primary_entropy"] = shannon_entropy(merged_df, rep_primary_cols)
merged_df["dem_primary_entropy"] = shannon_entropy(merged_df, dem_primary_cols)

In [39]:
# Final snippet of the cleaned and merged dataframe
merged_df.head(DISPLAY_ROWS)

,county,pri_dem_CLINTON,pri_dem_EDWARDS,pri_dem_OBAMA,pri_rep_HUCKABEE,pri_rep_MCCAIN,pri_rep_PAUL,pri_rep_ROMNEY,pri_rep_THOMPSON,gen_dem_OBAMA,gen_rep_MCCAIN,rep_primary_total,dem_primary_total,dem_general_total,rep_general_total,rep_primary_entropy,dem_primary_entropy
0,Adams,2829,145,829,1482,1492,75,104,67,4170,6914,3220,3803,4170,6914,0.992606,0.676711
1,Allen,7200,350,6801,4067,6813,848,738,323,19522,29940,12789,14351,19522,29940,1.137250,0.790505
2,Ashland,3913,153,2587,2687,3342,544,338,180,9300,15158,7091,6653,9300,15158,1.157562,0.766219
3,Ashtabula,13231,412,6375,2769,3818,682,329,214,25027,18949,7812,20018,25027,18949,1.162349,0.718008
4,Athens,7590,362,7396,1018,1771,408,108,71,20722,9742,3376,15348,20722,9742,1.146659,0.788402
5,Auglaize,3008,185,2085,1742,2615,490,282,124,6738,16414,5253,5278,6738,16414,1.179984,0.804801
6,Belmont,14204,932,4636,1283,2175,132,201,213,16302,15422,4004,19772,16302,15422,1.114922,0.721676
7,Brown,5487,277,1947,1945,2307,199,177,91,7503,12192,4719,7711,7503,12192,1.047978,0.709145
8,Butler,27599,589,22824,13758,19825,1179,1807,845,66030,105341,37414,51012,66030,105341,1.045325,0.743697
9,Carroll,3663,232,1731,1308,1839,275,166,103,6423,7097,3691,5626,6423,7097,1.147589,0.773533


In [40]:
# Save the cleaned and merged dataframe to CSV
out_dir = Path(OUTPUT_PATH)
out_dir.mkdir(parents=True, exist_ok=True)
merged_df.to_csv(OUTPUT_PATH + "OH.csv", index=False)